# Проект, Часть 1
## Царькова Анастасия, Скоробогатов Денис, Анвардинов Шариф

Для выполнения этого кода потребуются nltk (токенизация и стоп-слова), requests (http) и readability (выделение основного контента).

In [1]:
import re
import html
import json
import collections
import multiprocessing

from datetime import datetime, timedelta
from urllib.parse import urlparse

import requests
import readability

import nltk
import nltk.corpus

In [2]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /home/pyos/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /home/pyos/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### 1. HackerNews

**Задача**: выгрузить с HackerNews статьи со ссылками, комментарии, число лайков, даты публикации; перейти по каждой ссылке и сохранить основной контент статьи. Затем отфильтровать статьи, относящиеся к какому-то бренду.

In [3]:
HN_API_URL = 'https://hacker-news.firebaseio.com/v0/item/{}.json'
HN_API_MAX = 'https://hacker-news.firebaseio.com/v0/maxitem.json'

def traverse_comments(session, cur):
    '''
        Fetch & flatten the comment tree, starting with a given node.
    
        :param session: a requests session.
        :param cur: the tree root.

    '''
    for id in cur.get('kids', []):
        res = session.get(HN_API_URL.format(id)).json()
        if res is None:
            continue
        if not res.get('deleted'):
            yield res['text']
        for text in traverse_comments(session, res):
            yield text

def get_text(session, url):
    '''
        Fetch & extract main content from an external source.
        
        :param session: a requests session.
        :param url: the url of an article.

    '''
    try:
        return readability.Document(session.get(url).text).summary()
    except Exception:
        return 'could not fetch {}'.format(url)

Для ускорения краулить будем в 10 потоков. Разумеется, было бы оптимальней использовать asyncio, но это бы слишком сильно замедлило процесс разработки. Поскольку API у HackerNews оказался очень уж медленным, а времени довольно мало, сохранять статьи будем за последнюю неделю, а не за полгода.

In [4]:
def get_articles(i, w, n, output):
    '''
        Fetch HackerNews articles with comments.

        :param i: the id of the worker thread.
        :param w: total no. of worker threads. A thread will only check each `w`th id with phase `i`.
        :param n: the last item observed to actually exist on HackerNews.
        :param output: the queue to put the results into, or `None` when the thread exits.

    '''
    end = datetime.now() - timedelta(days=7)
    session = requests.Session()
    last_debug = n - i
    for id in range(n - i, 0, -w):
        try:
            res = session.get(HN_API_URL.format(id)).json()
            if res is None or res.get('type') != 'story' or res.get('deleted'):
                # HackerNews has a single id namespace for articles and comments;
                # in this loop, we're only interested in the former.
                continue
            time = datetime.fromtimestamp(res['time'])
            if last_debug - id > 5000:
                print('worker #{} - item #{} - {}'.format(i, id, time))
                last_debug = id
            if time < end:
                break
            output.put({
                'id': res['id'],
                'score': res['score'],
                'url': res.get('url') or None,
                'time': str(time),
                'text': res.get('text', '') if not res.get('url') else get_text(session, res['url']),
                'comments': list(traverse_comments(session, res)),
            })
        except Exception:
            pass
    print('worker #{} - done'.format(i))
    output.put(None)

In [5]:
w = 10
n = requests.get(HN_API_MAX).json()
print('Starting from {}'.format(n))

data = dict()
output = multiprocessing.Queue()
processes = [multiprocessing.Process(target=get_articles, args=(i, w, n, output)) for i in range(w)]
for p in processes:
    p.start()

try:
    while w:
        res = output.get()
        if res is None:
            w -= 1
        else:
            data[res.pop('id')] = res
    for p in processes:
        p.join()
finally:
    with open('hn-corpus.txt', 'w') as file:
        json.dump(data, file)

Starting from 15444717
worker #7 - item #15439520 - 2017-10-10 08:12:15
worker #4 - item #15439513 - 2017-10-10 08:10:52
worker #3 - item #15439424 - 2017-10-10 07:46:35
worker #8 - item #15439339 - 2017-10-10 07:24:01
worker #5 - item #15439492 - 2017-10-10 08:03:43
worker #6 - item #15439381 - 2017-10-10 07:33:39
worker #2 - item #15439125 - 2017-10-10 06:36:27
worker #1 - item #15439556 - 2017-10-10 08:22:44
worker #9 - item #15439308 - 2017-10-10 07:16:45
worker #0 - item #15439367 - 2017-10-10 07:28:46
worker #8 - item #15434109 - 2017-10-09 17:35:17
worker #4 - item #15434263 - 2017-10-09 17:53:47
worker #6 - item #15434021 - 2017-10-09 17:24:46
worker #3 - item #15433584 - 2017-10-09 16:29:48
worker #7 - item #15434100 - 2017-10-09 17:33:50
worker #9 - item #15433988 - 2017-10-09 17:21:52
worker #0 - item #15434187 - 2017-10-09 17:43:59
worker #5 - item #15434262 - 2017-10-09 17:53:43
worker #2 - item #15433665 - 2017-10-09 16:40:36
worker #1 - item #15434296 - 2017-10-09 17:57:

In [6]:
with open('hn-corpus.txt', 'r') as file:
    data = json.load(file)
len(data)

6890

In [7]:
min(v['time'] for v in data.values())

'2017-10-03 22:39:13'

Прежде чем что-то делать, из статей и комментариев следует выкинуть HTML-разметку и капитализацию.

In [8]:
tags = re.compile(r'(<[^>]*>|\s)+')
for k, v in data.items():
    v['text'] = html.unescape(tags.sub(' ', v['text'].lower()))
    v['comments'] = list(map(lambda x: html.unescape(tags.sub(' ', x.lower())), v['comments']))

In [9]:
with open('hn-corpus-stripped.txt', 'w') as file:
    json.dump(data, file)

Бренд выберем, например, Google. Здесь есть проблема: как понять, что статья относится именно к нему, а не, скажем, мельком упоминает его в контексте новости о каком-нибудь конкуренте? Задача выглядит довольно сложной, так что решать ее не будем, а просто выберем статьи с хоть одним упоминанием.

In [10]:
brand_data = {k: v for k, v in data.items() if re.search(r'\bgoogle\b', v['text'])}

### 2. Исследование выборки

**Сколько постов в собранном корпусе?**

In [11]:
len(brand_data)

1311

**Из каких источников состоит корпус?**

In [12]:
def hostname(url):
    host = urlparse(url).hostname
    if host.startswith('www.'):
        return host[4:]
    return host

hosts = collections.Counter(hostname(v['url']) for k, v in brand_data.items() if v['url'])
for host, count in hosts.most_common(30):
    print('{: >30} {}'.format(host, count))

                    medium.com 83
                techcrunch.com 35
                   nytimes.com 26
                hackernoon.com 23
               arstechnica.com 19
                     wired.com 18
               theguardian.com 17
            securityaffairs.co 16
               theatlantic.com 15
                  betanews.com 14
                 bloomberg.com 12
             theregister.co.uk 12
                       eff.org 12
                        goo.gl 10
                     zdnet.com 9
          technologyreview.com 9
               venturebeat.com 9
             bitsandscrews.com 8
                        qz.com 8
              en.wikipedia.org 8
               fastcompany.com 7
                      cnbc.com 7
           businessinsider.com 7
                   blog.google 7
                   twitter.com 7
                      cnet.com 7
             blog.alexellis.io 7
                  politico.com 6
        medium.facilelogin.com 6
                    nature.co

**Какие слова (не считая стоп-слов) встречаются чаще всего?**

Небольшое отступление: у nltk очень странное поведение по отношению к стоп-словам. Когда строили их список, апостроф явно считали разделителем. Например, там есть такие "огрызки", как `wasn` (очевидно, от `wasn't`):

In [13]:
stopwords = nltk.corpus.stopwords.words('english')
stopwords[-10:]

['ma',
 'mightn',
 'mustn',
 'needn',
 'shan',
 'shouldn',
 'wasn',
 'weren',
 'won',
 'wouldn']

`nltk.word_tokenize` же осведомлен о правиле `<P> not -> <P>n't` и выделяет `n't` отдельным токеном:

In [14]:
nltk.word_tokenize("mustn't")

['must', "n't"]

Поэтому при токенизации апостроф следует заменить на пробел.

In [17]:
def get_word_counts(brand_data, stopwords):
    texts = []
    for v in brand_data.values():
        texts.append(v['text'])
        texts.extend(v['comments'])
    return collections.Counter(w for t in texts for w in nltk.word_tokenize(t.replace("'", " ")) if w.isalpha() and w not in stopwords)

c = get_word_counts(brand_data, stopwords)
for w, n in c.most_common(20):
    print('{: >20} {}'.format(w, n / sum(c.values())))

                like 0.004919924727830389
                 one 0.004555320505713497
              google 0.004259172586524675
              people 0.004081335016961537
                 new 0.003988323735809269
                data 0.0036371131381783034
                also 0.003428023778148004
               would 0.0033037607065285733
                time 0.0032844143600489015
             element 0.0032591152915754846
                 use 0.0032137257863731777
                 get 0.0030396086680561313
                make 0.0024488010101769223
                even 0.0024130846782144514
               could 0.0023185852165637467
                work 0.0022560816356294224
                need 0.0021615821739787177
               using 0.002159349903231063
                 way 0.0021310744737607734
               first 0.002082708607561594


Слова довольно-таки негуглоспецифичные.

**Из всех собранных ссылок, сколько ведут на блоги и новостные издания, а сколько – на github и другие неновостные издания?**

Сразу же возникает вопрос &mdash; как отличать "новостные издания" от "неновостных изданий"? Для некоторых доменов, таких как `medium.com`, корректный класс известен, но в общем случае сайты отличить довольно сложно. Некоторые люди к тому же, например, ведут блоги на GitHub, используя GitHub Pages или даже просто Markdown файлы &mdash; кроме как вручную это не классифицировать.

In [19]:
KNOWN_NEWS = [
    'businessinsider.com', 'forbes.com', 'wired.com', 'bloomberg.com',
    'techcrunch.com', 'theverge.com', 'arstechnica.com', 'theguardian.com',
    'engadget.com', 'washingtonpost.com', 'venturebeat.com', 'reuters.com',
]

KNOWN_BLOGS = [
    'medium.com', 'hackernoon.com', '.blogspot.com',
]

def host_category(host):
    if 'news.' in host:
        return 'news'
    if 'blog.' in host:
        return 'blog'
    for k in KNOWN_NEWS:
        if host == k or (k.startswith('.') and host.endswith(k)):
            return 'news'
    for k in KNOWN_BLOGS:
        if host == k or (k.startswith('.') and host.endswith(k)):
            return 'blog'

collections.Counter(host_category(h) for h in hosts.elements())

Counter({None: 927, 'blog': 207, 'news': 157})